In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [5]:
from time_series import revert

In [6]:
qb2 = pd.read_csv('../data/dashboard/qb/qb_smoothed.csv')
rb2 = pd.read_csv('../data/dashboard/rb/rb_smoothed.csv')
wr2 = pd.read_csv('../data/dashboard/wr/wr_smoothed.csv')

qb2.rename(columns={'player_display_name': 'player'}, inplace=True)
rb2.rename(columns={'player_display_name': 'player'}, inplace=True)
wr2.rename(columns={'player_display_name': 'player'}, inplace=True)

qb2['position'] = 'QB'
rb2['position'] = 'RB'
wr2['position'] = 'WR'

In [7]:
qb2 = qb2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})
rb2 = rb2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})
wr2 = wr2.rename(columns={'predicted_smoothed_salary': 'predicted_salary'})

In [8]:
reverting = revert(qb2, position = 'QB', year = 2024)
qb = reverting.unnormalize('smooth')

reverting = revert(rb2, position = 'RB', year = 2024)
rb = reverting.unnormalize('smooth')

reverting = revert(wr2, position = 'WR', year = 2024)
wr = reverting.unnormalize('smooth')

In [9]:
qb.sample(2)

,Unnamed: 0,player,year_signed,games,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_fumbles,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,rushing_yards,rushing_tds,fantasy_points,salary_per_year_x,mean_adj_salary_per_year,med_adj_salary_per_year,salary_per_year_y,smoothed_salary_per_year,predicted_median_salary,predicted_mean_salary,predicted_salary,position,SAD,smoothed_apy,reverted_pred_salary
21,38,C.J. Beathard,2024.0,6.0,40.0,53.0,349.0,1.0,0.0,6.0,0.0,19.0,5.580715,0.0,1.029499,0.161698,35.0,0.0,21.46,1.310,-0.237972,0.655778,1.310,-0.368248,5.085180,0.062948,0.035681,QB,3.952081,2.765347,2.906362
8,16,Skylar Thompson,2024.0,8.0,78.0,150.0,754.0,2.0,5.0,10.0,1.0,38.0,-43.411161,1.0,0.589063,-0.021004,24.0,0.0,28.56,0.225,-0.392348,-1.295817,0.225,-0.642787,2.344567,-0.098909,-0.277426,QB,3.952081,2.765347,1.668938


In [10]:
qb_best_features = ['player', 'position','passing_tds', 'passing_epa','rushing_yards', 'salary_per_year_y', 'reverted_pred_salary']
rb_best_features = ['player', 'position', 'rushing_yards', 'target_share', 'receiving_yards_after_catch','salary_per_year','reverted_pred_salary']
wr_best_features = ['player', 'position', 'receiving_yards', 'target_share', 'receiving_first_downs', 'salary_per_year', 'reverted_pred_salary']

qb = qb[qb_best_features]
rb = rb[rb_best_features]
wr= wr[wr_best_features]

rename_cols = lambda df: df.rename(columns={df.columns[2]: 'feature_1', df.columns[3]: 'feature_2', df.columns[4]: 'feature_3', 'player':'name', 'salary_per_year_y': 'actual_salary', 'reverted_pred_salary': 'predicted_salary', 'salary_per_year': 'actual_salary'})
qb = rename_cols(qb)
rb = rename_cols(rb)
wr = rename_cols(wr)

full  = pd.concat([qb, rb, wr], ignore_index=True)
full.sample(5)

,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
52,Chuba Hubbard,RB,902.0,0.086614,269.0,8.300,5.020331
143,A.T. Perry,WR,246.0,0.070588,10.0,0.225,3.032969
63,Ameer Abdullah,RB,89.0,0.049383,88.0,1.850,1.370950
65,Aaron Jones,RB,882.0,0.120482,297.0,7.000,6.697747
1,Tua Tagovailoa,QB,27.5,103.495877,84.5,53.100,36.620123


In [11]:
numerics = ['feature_1', 'feature_2', 'feature_3', 'actual_salary', 'predicted_salary']
full[numerics] = full[numerics].round(2)
full = full[full['name'] != 'Russell Wilson']
full = full[full['name'] != 'Tee Higgins']
print(full.shape)

## undervalued 
full[(full.predicted_salary - full.actual_salary) > 3]

(226, 7)


,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
9,Desmond Ridder,QB,7.0,-14.43,128.5,0.98,6.35
22,Joshua Dobbs,QB,13.0,-80.75,421.0,2.25,10.33
26,Baker Mayfield,QB,34.0,67.00,194.0,33.33,43.27
29,Jake Browning,QB,12.0,22.30,127.0,0.97,16.03
62,Raheem Mostert,RB,1045.0,0.07,216.0,4.13,8.21
107,Tim Patrick,WR,734.0,0.17,39.0,1.30,9.47
147,Rashid Shaheed,WR,719.0,0.14,27.0,3.09,8.11
192,Mike Evans,WR,1450.0,0.25,61.0,20.50,28.22
207,Keelan Doss,WR,33.0,0.16,1.0,1.06,4.07
224,Josh Reynolds,WR,740.0,0.12,39.0,4.50,11.06


In [12]:
## overvalued 
full[(full.actual_salary - full.predicted_salary) > 10]

,name,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
0,Jared Goff,QB,31.5,104.48,48.5,53.0,41.61
1,Tua Tagovailoa,QB,27.5,103.50,84.5,53.1,36.62
2,Dak Prescott,QB,33.5,87.49,257.5,60.0,41.26
7,Kirk Cousins,QB,24.5,33.91,61.5,45.0,31.40
117,Darnell Mooney,WR,414.0,0.15,20.0,13.0,2.57


In [13]:
### joining in height, weight, and age

players = pd.read_csv('../data/raw/players.csv')

full = full.merge(
    players[['display_name', 'birth_date', 'height', 'weight']], 
    how='left', 
    left_on='name', 
    right_on='display_name'
)


full = full.drop_duplicates(subset='name', keep='first')
full['birth_date'] = pd.to_datetime(full['birth_date'], errors='coerce')
today = pd.Timestamp.today()
full['age'] = (today - full['birth_date']).dt.days // 365

full = full.drop(columns=['display_name'])
full = full.drop(columns=['birth_date'])

full = full[['name', 'height', 'weight', 'age', 'position', 'feature_1', 'feature_2', 'feature_3', 'actual_salary', 'predicted_salary']]
print(full.shape)
full.sample(5)

(226, 10)


,name,height,weight,age,position,feature_1,feature_2,feature_3,actual_salary,predicted_salary
66,Aaron Jones,69.0,208.0,30.0,RB,882.0,0.12,297.0,7.00,6.70
140,Isaiah Hodgins,76.0,209.0,26.0,WR,230.0,0.08,12.0,1.03,1.81
57,James Conner,73.0,233.0,29.0,RB,1040.0,0.09,221.0,9.50,7.20
91,Josh Jacobs,70.0,220.0,27.0,RB,805.0,0.14,293.0,12.00,4.88
48,Cam Akers,71.0,212.0,25.0,RB,167.0,0.06,78.0,1.18,1.06


In [14]:
full.to_csv('../data/dashboard/full.csv')

In [15]:
full.to_csv('../final_dashboard_data.csv')